<a href="https://colab.research.google.com/github/diana-07/Diana_tests_DL/blob/main/project_DL_Diana_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# #eliminar pasta dataset caso seja necessário
# import shutil
# import os

# os.chdir('/content')
# print(os.getcwd())

# # Caminho para a pasta a apagar
# dir_path = "dataset"

# # Só apaga se existir
# if os.path.exists(dir_path):
#     shutil.rmtree(dir_path)
#     print(f"Pasta '{dir_path}' apagada com sucesso.")
# else:
#     print(f"Pasta '{dir_path}' não existe.")


In [2]:
import os
import shutil
import sys

#IMPORTANTE: BAIXAR ANTES O FICHEIRO prepare_data.py

# 1. Remover pasta 'dataset' se já existir
if os.path.exists("dataset"):
    shutil.rmtree("dataset")
    print("Pasta 'dataset' removida.")

# 2. Clonar o repositório com checkout parcial
os.system("git clone --filter=blob:none --no-checkout https://github.com/THETIS-dataset/dataset.git")

# 3. Entrar na pasta 'dataset' e configurar checkout parcial
os.chdir("dataset")
os.system("git sparse-checkout init --cone")
os.system("git sparse-checkout set VIDEO_Skelet3D")
os.system("git checkout")
print("Clone e checkout parcial concluídos.")

# 4. Voltar ao diretório anterior
os.chdir("..")


# 5. Procurar prepare_data.py fora da pasta 'dataset'
nome_arquivo = "prepare_data.py"
caminho_encontrado = None

for raiz, dirs, arquivos in os.walk("."):
    if nome_arquivo in arquivos and "dataset" not in raiz:
        caminho_encontrado = os.path.join(raiz, nome_arquivo)
        break

# 6. Mover o prepare_data.py para dentro de 'dataset'
if caminho_encontrado:
    destino_final = os.path.join("dataset", nome_arquivo)
    shutil.move(caminho_encontrado, destino_final)
    print(f"{nome_arquivo} movido para a pasta 'dataset'.")
else:
    print(f"Arquivo {nome_arquivo} não encontrado fora da pasta 'dataset'.")

# 7. Adicionar 'dataset' ao sys.path para permitir importações
dataset_path = os.path.abspath("dataset")
if dataset_path not in sys.path:
    sys.path.append(dataset_path)

# 8. Agora já podes importar a função
try:
    from prepare_data import collect_by_players
    from prepare_data import pad_or_truncate_keypoints
    from prepare_data import extract_player_id
    print("Importação bem-sucedida.")
except ImportError as e:
    print("Erro ao importar função:", e)






Pasta 'dataset' removida.
Clone e checkout parcial concluídos.
prepare_data.py movido para a pasta 'dataset'.
Importação bem-sucedida.


In [3]:
import os
import cv2
import torch
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from sklearn.model_selection import train_test_split
import random
import pickle

# from prepare_data import pad_or_truncate_keypoints
# from prepare_data import extract_player_id


In [4]:
!pip uninstall -y mediapipe-silicon
!pip uninstall -y mediapipe
!pip uninstall -y protobuf
!pip install protobuf==3.20.3  # Specific version that often works well with MediaPipe
!pip install mediapipe

Found existing installation: mediapipe 0.10.21
Uninstalling mediapipe-0.10.21:
  Successfully uninstalled mediapipe-0.10.21
Found existing installation: protobuf 4.25.7
Uninstalling protobuf-4.25.7:
  Successfully uninstalled protobuf-4.25.7
  Using cached protobuf-3.20.3-py2.py3-none-any.whl.metadata (720 bytes)
Using cached protobuf-3.20.3-py2.py3-none-any.whl (162 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.17.1 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.


  Using cached mediapipe-0.10.21-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (9.7 kB)
  Using cached protobuf-4.25.7-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
Using cached mediapipe-0.10.21-cp311-cp311-manylinux_2_28_x86_64.whl (35.6 MB)
Using cached protobuf-4.25.7-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.7 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.7 which is incompatible.


In [5]:
import mediapipe as mp


In [6]:
%cd dataset
!python prepare_data.py

/content/dataset
CUDA available: True
Device name: Tesla T4
['backhand_slice', 'forehand_openstands', 'forehand_flat', 'kick_service', 'flat_service', 'backhand_volley', 'backhand', 'smash', 'slice_service', 'forehand_slice', 'forehand_volley', 'backhand2hands']
Fim da execução do prepare_data.py


In [7]:

#extract the keypoints AND pad/truncate

root_dir = "npy_videos"

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False) # This allows MediaPipe to track pose across frames, potentially leading to more stable and accurate keypoint estimations.

all_keypoints = {}

# Top-level loop with a single progress bar
action_folders = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
for action_folder in tqdm(action_folders, desc="Processing actions"):
    action_path = os.path.join(root_dir, action_folder)
    all_keypoints[action_folder] = {}

    video_files = sorted([f for f in os.listdir(action_path) if f.endswith(".npy")])
    for video_file in video_files:  # No tqdm here
        video_path = os.path.join(action_path, video_file)
        try:
            sample = np.load(video_path)
            video_keypoints = []

            for frame in sample:
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = pose.process(frame_rgb)

                if results.pose_landmarks:
                    keypoints = [[lm.x, lm.y, lm.z] for lm in results.pose_landmarks.landmark]
                else:
                    keypoints = np.zeros((33, 3)).tolist()

                video_keypoints.append(keypoints)

            video_keypoints = np.array(video_keypoints)
            video_keypoints = pad_or_truncate_keypoints(video_keypoints, target_length=120)
            all_keypoints[action_folder][video_file] = video_keypoints





        except Exception as e:
            pass  # Suppress per-video error printing for a clean log

# Final summary
print("\n Summary of extracted keypoints:")
for action, videos in all_keypoints.items():
    print(f"- {action}: {len(videos)} videos processed.")

# Jsut to see the calues for the first one
first_action = list(all_keypoints.keys())[0]
first_video = list(all_keypoints[first_action].keys())[0]
print(f"Keypoints for the first frame of {first_video} in {first_action}:")
print(all_keypoints[first_action][first_video][0])

print('----')

frame_idx = 10  # ou qualquer outro frame entre 0 e 119 (porque truncas para 120)
print(f"Keypoints para o frame {frame_idx} do vídeo {first_video} ({first_action}):")
print(all_keypoints[first_action][first_video][frame_idx])

print()

# Por exemplo, mostrar todos os pontos do frame 10
for i, (x, y, z) in enumerate(all_keypoints[first_action][first_video][frame_idx]):
    print(f"  Ponto {i:2d}: x={x:.3f}, y={y:.3f}, z={z:.3f}")



Processing actions: 100%|██████████| 12/12 [23:27<00:00, 117.28s/it]


 Summary of extracted keypoints:
- backhand: 97 videos processed.
- backhand2hands: 107 videos processed.
- backhand_slice: 100 videos processed.
- backhand_volley: 103 videos processed.
- flat_service: 96 videos processed.
- forehand_flat: 110 videos processed.
- forehand_openstands: 101 videos processed.
- forehand_slice: 97 videos processed.
- forehand_volley: 93 videos processed.
- kick_service: 109 videos processed.
- slice_service: 100 videos processed.
- smash: 104 videos processed.
Keypoints for the first frame of p10_backhand_skelet3D_s2.npy in backhand:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
----
Keypoints para o frame 10 d

In [8]:
#Organize videos by player and split player IDs

# Organize videos by player
player_video_map = defaultdict(list)
for action, videos in all_keypoints.items():
    for video_file in videos:
        player_id = extract_player_id(video_file)
        player_video_map[player_id].append((action, video_file))

# Split player IDs
players = list(player_video_map.keys())
random.shuffle(players)

train_end = int(len(players) * 0.7)
val_end = int(len(players) * 0.85)

train_players = players[:train_end]
val_players = players[train_end:val_end]
test_players = players[val_end:]

train_set = collect_by_players(train_players, player_video_map, all_keypoints)
val_set = collect_by_players(val_players,  player_video_map, all_keypoints)
test_set = collect_by_players(test_players,  player_video_map, all_keypoints)

print(f"Train: {len(train_set)}, Val: {len(val_set)}, Test: {len(test_set)}")

Train: 822, Val: 183, Test: 212


In [17]:
# Guardar cada conjunto em ficheiro .pkl
with open('train_set.pkl', 'wb') as f:
    pickle.dump(train_set, f)
with open('val_set.pkl', 'wb') as f:
    pickle.dump(val_set, f)
with open('test_set.pkl', 'wb') as f:
    pickle.dump(test_set, f)

In [10]:
#  Usar biblioteca MMAction2

# guardar os pesos treinados no dataset Kinetics400 numa variavel
#pretrained='https://download.openmmlab.com/mmaction/stgcn/stgcn_50-pyslowfast-kinetics400-pretrained.pth'

#modelo
# Criar automaticamente o modelo ST-GCN-50
# carrega pesos pre-treinados
# considera o número de classes do projeto
# model = dict(
#     type='STGCN',
#     backbone=dict(
#         type='STGCN',
#         graph_cfg=dict(layout='openpose', mode='spatial'),
#         pretrained= pretrained # Checkpoint pré-treinado
#     ),
#     cls_head=dict(type='STGCNHead', num_classes=12, in_channels=256)
# )



In [13]:
# definir os dados de treino, validação e teste
# files no formato .pkl com keypoints: (N, C, T, V, M) e labels.

# data = dict(
#     train=dict(ann_file='train_set.pkl'),
#     val=dict(ann_file='val_set.pkl'),
#     test=dict(ann_file='test_set.pkl')
# )

{'train': {'ann_file': 'train_set.pkl'},
 'val': {'ann_file': 'val_set.pkl'},
 'test': {'ann_file': 'test_set.pkl'}}

In [14]:
# Clonar e entrar no diretório
!git clone https://github.com/open-mmlab/mmaction2.git
%cd mmaction2

# Instalar dependências
!pip install -r requirements/build.txt
!pip install -v -e .

# Verifica se tools/train.py existe
!ls tools/train.py



Cloning into 'mmaction2'...
remote: Enumerating objects: 22864, done.
remote: Total 22864 (delta 0), reused 0 (delta 0), pack-reused 22864 (from 1)
Receiving objects: 100% (22864/22864), 69.65 MiB | 26.06 MiB/s, done.
Resolving deltas: 100% (16122/16122), done.
/content/dataset/mmaction2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 111.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 853.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta

Using pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
Obtaining file:///content/dataset/mmaction2
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-04rrzil7/mmaction2.egg-info
  writing /tmp/pip-pip-egg-info-04rrzil7/mmaction2.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-04rrzil7/mmaction2.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-pip-egg-info-04rrzil7/mmaction2.egg-info/requires.txt
  writing top-level names to /tmp/pip-pip-egg-info-04rrzil7/mmaction2.egg-info/top_level.txt
  writing manifest file '/tmp/pip-pip-egg-info-04rrzil7/mmaction2.egg-info/SOURCES.txt'
  reading manifest file '/tmp/pip-pip-egg-info-04rrzil7/mmaction2.egg-info/SOURCES.txt'
  reading manifest template 'MANIFEST.in'
  adding license file 'LICENSE'
  writing manifest file '/tmp/pip-pip-egg-info-04rrzil7/mmaction2.egg-info/SOURCES.txt'
  Preparing metadata (setup.py) ... done
  Running setup.py dev

In [16]:
# Isto cria o ficheiro automaticamente no sítio certo
# criar ficheiro para configurar o treino do modelo
# config_code = """
# model = dict(
#     type='STGCN',
#     backbone=dict(
#         type='STGCN',
#         graph_cfg=dict(layout='openpose', mode='spatial'),
#         pretrained='https://download.openmmlab.com/mmaction/stgcn/stgcn_50-pyslowfast-kinetics400-pretrained.pth'
#     ),
#     cls_head=dict(type='STGCNHead', num_classes=12, in_channels=256)
# )

# dataset_type = 'PoseDataset'
# data = dict(
#     videos_per_gpu=64,
#     workers_per_gpu=2,
#     train=dict(type=dataset_type, ann_file='dataset/train_set.pkl', data_prefix=''),
#     val=dict(type=dataset_type, ann_file='dataset/val_set.pkl', data_prefix=''),
#     test=dict(type=dataset_type, ann_file='dataset/test_set.pkl', data_prefix='')
# )

# optimizer = dict(type='Adam', lr=0.0003, weight_decay=0.0005)
# optimizer_config = dict(grad_clip=None)

# lr_config = dict(policy='CosineAnnealing', min_lr=0)
# total_epochs = 10
# checkpoint_config = dict(interval=10)

# evaluation = dict(
#     interval=1,
#     metrics=['top_k_accuracy', 'mean_class_accuracy'],
#     metric_options=dict(top_k_accuracy=dict(topk=(1, 3)))
# )

# log_config = dict(
#     interval=20,
#     hooks=[dict(type='TextLoggerHook')]
# )

# work_dir = './work_dirs/stgcn_50_tennis'
# """

# # Gravar para o ficheiro na pasta configs/stgcn/
# with open('dataset/mmaction2/configs/stgcn/stgcn_50-tennis.py', 'w') as f:
#     f.write(config_code)



FileNotFoundError: [Errno 2] No such file or directory: 'dataset/mmaction2/configs/stgcn/stgcn_50-tennis.py'

In [18]:
# Isto cria o ficheiro automaticamente no sítio certo
# criar ficheiro para configurar o treino do modelo
config_code = """
_base_ = '../../configs/skeleton/stgcn/stgcn_8xb16-joint-u100-80e_ntu60-xsub-keypoint-2d.py'

model = dict(
    cls_head=dict(num_classes=12))

dataset_type = 'PoseDataset'
data = dict(
    videos_per_gpu=64,
    workers_per_gpu=2,
    train=dict(type=dataset_type, ann_file='dataset/train_set.pkl'),
    val=dict(type=dataset_type, ann_file='dataset/val_set.pkl'),
    test=dict(type=dataset_type, ann_file='dataset/test_set.pkl'))

optimizer = dict(type='Adam', lr=0.0003, weight_decay=0.0005)
total_epochs = 10
"""

# Gravar para o ficheiro na pasta configs/stgcn/
with open('configs/stgcn/stgcn_50-tennis.py', 'w') as f:
    f.write(config_code)




FileNotFoundError: [Errno 2] No such file or directory: 'configs/stgcn/stgcn_50-tennis.py'

In [12]:
!python tools/train.py configs/stgcn/stgcn_50-tennis.py

SyntaxError: invalid syntax (<ipython-input-12-49585970d48c>, line 1)